# Top End Assembly Vibration Study-- MTCamHexapod Telemetries

##### As described in OBS-467, we have been noticing strong vibrations on the Top End Assembly (TEA) of the Simonyi Survey Telescope (SST). The first time we saw these vibrations was on March 9th (Saturday) and on March 10th (Sunday).

##### Accordingly to SITCOM-1285, on March 15th (Friday), we ran soak tests using the Rotator at different configurations. We want to investigate any possible vibrations during that period to confirm the hypothesis that the Rotator could be causing this vibration.

#### Analysis details 

##### The idea is to explore possible sources and /or components affected by the vibrations. This analysis focusses on MTCam hexapods and vibrations it may/may not have experienced during the above event. 

##### Note: The natural frequency of both Camera and M2 hexapods is around 12 Hz. 
##### Douglas R Neill 2024 04 18
##### Regarding Bruno SITCOM-1285.
##### It is unlikely that the large accelerations of 1.2 m/s2 could be produced by anything other than the TMA drives. This contradicts the statements that the TMA was locked out at this time. The TEA has very low damping and a vibration amplification of 20 would not be unexpected. If the excitation is near a natural frequency, this could be produced by an excitation of only .06 m/s2 somewhere else. The natural frequencies of the camera and M2 on their hexapods are both around 12 Hz. This large acceleration is reported to be on the M2 surrogate, but I was later informed in a meeting that it is unclear where these accelerometers actually are.
##### The rotator is shown to have oscillation of order +/- .2 e-3 deg. The CG of ComCam is supposed to be off from the rotational axis of order 10mm. This rotation is consistent with the effects of transverse vibration being applied to the camera thru the TEA.  A transverse vibration would result in a small rotation. This effect can also be produced by a torque in the system produced by a wire, interference etc. I will just call it an out of balance.
##### Page 12. Both the SST M2 surrogate and the top end ring accelerometers show large accelerations. Knowing the frequency of these vibrations would be useful. The natural frequency of the hexapods are both around 12 Hz. This amplifies vibrations near this frequency, and attenuate them for frequencies greater than ~18 hz. If the excitation came from the camera or rotator and is around 12 Hz, it would excite the M2. If it is greater than 18 Hz it would be almost impossible for the camera or rotator to be producing this vibration. The strange rotations of the rotor are difficult to unexplained unless they are the result of a transverse impulse as described above.
##### If the excitation is from an external source, we would still expect to see some variations if the rotation is on or off. The rotator control system would attempt to counteract the vibration. The figures on Pages 18 and 19 show negligible change in the spider, M2 and top end vibrations with the rotator on or off.
##### Page 19 show M2 accelerations at frequencies of ~50 Hz. A frequency this high cannot come from the camera or rotator. It would be attenuated by their natural frequencies. The small vibration at ~12 hz is consistent with the M2 natural frequency. The ~50Hz may just be an error produced by 50 Hz power. Later it is theorized this is a data sampling error.
##### Bruno message at 3:50. Important clarification.  The maximum TMA acceleration is ~0.06G or 0.6 m/s2. The message says 0.6G. Easy to get this confused.
##### Bruno message 4:01 The figure shows spikes in the M2 IMS in the y direction corresponding with rotation in the rotator. This is consistent with an excitation of the top end in the y direction (rotation in the elevation axis) and a camera CG off set in the X direction. This would be true if the excitation was from an internal source, TMA drives or an internal source to the top end. However, there is nothing in the top end that can make this order of vibrations and the TMA drive system was off.
##### Bruno message 4:49. The motions of the M2 IMS are consistent with rotations in elevation. As the gravity vector changes the displacement changes between Y and Z. the X displacement is almost zero. The rotator angle is consistent with the elevation torque if there is an out of balance in the X. If we rotated the camera 90 degrees, we would expect this behavior to change substantially.
##### Bruno message 4:49 second plot. I am suspecting that we are seeing two manifestations of the same issue. Perhaps there is a larger out of balance in ComCam than is expected. This would produce rotations from Y vibration accelerations. The rotator might also have trouble holding its position.
##### After reviewing this message stream, I find it unlikely that the vibrations are produced by either the rotator or ComCam. They appear to be the recipients of the vibration and not the cause. I cannot see anything that could cause these levels of accelerations other than the TMA drives which are reported to be off.
##### It appears that all the tests were with the rotator at zero. I suspect that there is a large out of balance in ComCam which is causing a correlation between y displacement and rotation. This can be demonstrated by rotating near +/-90 degrees. The behavior should change significantly. If the out of balance is initially along the X direction it will now be along the Y direction. We would then see a correlation between X motion and rotation. The X motion is not produced by the elevation motion. It is only produced by Azimuth rotations away from zenith pointing.


### Import modules

In [1]:
%matplotlib inline
%matplotlib widget
from datetime import datetime
import time
from astropy.time import Time
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.signal import detrend

from lsst_efd_client import EfdClient


### Declaration of User-defined Functions

In [2]:
async def get_efd_data(
    efd_client: EfdClient,
    time_start: Time,
    time_end: Time,
) -> DataFrame:
    """Get the EFD data.

    Parameters
    ----------
    efd_client : `lsst_efd_client.EfdClient`
        EFD client.
    time_start : `astropy.time.Time`
        Start time.
    time_end : `astropy.time.Time`
        End time.

    Returns
    -------
    `pandas.core.frame.DataFrame`
        Queried data.
    """
    data = await efd_client.select_time_series(
    "lsst.sal.MTHexapod.application", 
    fields=["demand", "position", "timestamp"], 
    start=time_start, 
    end=time_end)

    return data

In [3]:
data.position

NameError: name 'data' is not defined

In [5]:
def plot_position_and_frequency(
    data: DataFrame,
    freq_range: tuple[float] | None = None,
    snr: int | None = None,
    threshold: float = 1e-3, 
    n: int 
) -> None:
    """Plot the position and frequency data.

    Parameters
    ----------
    data : `pandas.core.frame.DataFrame`
        EFD data.
    freq_range : `tuple` [`float`], optional
        Frequency range (low, high) to monitor in Hz. If None, do not
        check. (the default is None)
    snr : `int`, optional
        Signal-to-noise ratio when checking the frequency range.
    threshold : `float`, optional
        Threshold to check the detrend data is good or not.
    n : in order (X, Y, Z, U, V, W)
    """
    n
    actual_position = np.array(data.position[n])
    demand_position = np.array(data.demandPosition[n])
    displacement_error = actual_position - demand_position

    timestamps = np.array(data.timestamp)
    timestamps = timestamps - timestamps[0]

    detrend_displacement_error = detrend(displacement_error, type="linear");

    is_detrend_data_ok = np.all(np.abs(detrend_displacement_error) <= threshold)
    print(f"{is_detrend_data_ok=}")

    data_fft_position, frequency_fft_position = calculate_fft(
        detrend_displacement_error,
        timestamps[1] - timestamps[0],
    )

    fig, ax = plt.subplots(1, 3, figsize=(12, 4))
    ax[0].plot(timestamps, actual_position, "b")
    ax[0].plot(timestamps, demand_position, "r--")
    ax[0].set_xlabel("Time (s)")
    ax[0].set_ylabel("Position (um)")
    ax[0].legend(["Actual", "Demand"])
    
    ax[1].plot(timestamps, displacement_error, "b")
    ax[1].plot(timestamps, detrend_displacement_error, "r-")
    ax[1].set_xlabel("Time (s)")
    ax[1].set_ylabel("Displacement (um)")
    ax[1].legend(["Displacement Error", "Detrend Displacement Error"])

    ax[2].plot(frequency_fft_position, data_fft_position[0:len(actual_position)//2])
    ax[2].set_xlabel("Frequency (Hz)")

    plt.show()

    if (freq_range is not None) and (snr is not None):
        freq = check_low_freq(
            detrend_displacement_error,
            timestamps[1] - timestamps[0],
            freq_range[0],
            freq_range[1],
            snr,
        )

        if (freq == 0.0) or (not is_detrend_data_ok):
            print("No frequency peak found in the range.")
        else:
            print(f"The maximum frequency peak found is {freq:.2f} Hz")

SyntaxError: non-default argument follows default argument (3694617241.py, line 6)